# Etapa 1 del proyecto. La tarea de regresión: modelos polinomiales y regularizados

En este proyecto se nos da un enunciado del problema. En primera medida se analiza y se resaltan las partes mas importantes para el desarrollo del mismos, identidicando el problema real, el objetivo a conseguir(variable objetivo) y los datos que tenemos(variables independientes).

Actualmente, estamos ante la presencia de una variedad de fenómenos que están afectando el medio ambiente y que se reflejan en el calentamiento global, la **contaminación del aire** y la pérdida de biodiversidad, entre otros. Establecer políticas que ataquen sus causas y el desarrollo de planes de **gestión de riesgos que mitiguen sus efectos** y permitan accionar oportunamente es uno de los grandes retos a los que nos enfrentamos como sociedad.

Uno de estos problemas está relacionado con el cambio climático, el cual se refiere tanto al calentamiento global provocado por la **emisión de gases de efecto invernadero,** como al aumento de los desastres ambientales inducidos por dicho calentamiento. En particular, en el caso del **dióxido de carbono (CO2)**, aunque es necesario para la vida, **su exceso** de concentración **contribuye significativamente al efecto invernadero**, que eleva la temperatura del planeta y desequilibra el ciclo natural. Una manera de reducir las emisiones de CO2 es fomentar planes de movilidad sostenible, la cual se refiere a la capacidad de hacer traslados, de personas o mercancías, con seguridad y sin afectar la salud ni los ecosistemas. Apoyar e incentivar medidas de movilidad sostenible, como el **uso de vehículos eléctricos y bicicletas**, contribuiría en gran medida a paliar este problema.

En este contexto, el problema que se va a abordar es:
- **Construir un modelo predictivo** que permita 
- **Determinar la demanda** sobre el 
- **Uso de un sistema de alquiler de bicicletas**. 

Este conocimiento puede dar soporte para mejorar el servicio y conocer los factores que inciden en su eficiencia.

Teniendo encuenta entonces este simple analisis y revisando el archivo .csv podemos determinar que:
- Variable objetivo o dependiente 'y' (Demanda o Cantidad de bicicletas rentadas)
- Variables independientes 'x' (season, weekday, weathersit, temp, atemp, hum, windspeed, time_of_day)

Realizaremos ahora la importación de las librerías necesarias para el desarrollo del proyecto

In [4]:
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, KFold

#Aunque el proyecto no lo pide explicitamente intentaré aplicar un escalador dado el caso que que se necesite o no, ya depende del análisis que se realice mas adelante
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import Lasso
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from importlib.metadata import version

print(f"Versión de Scikit-learn: {version('scikit-learn')}")
print(f"Versión de Pandas: {version('pandas')}")

Versión de Scikit-learn: 1.8.0
Versión de Pandas: 2.3.3


# 1. **Exploración y perfilamiento de los datos**

Como primera medida vamos a renombrar el archivo csv ya que viene como **"ETYT2_6WS-GwUXGPSWerJQ_c011f04f69254a4087d4396a2e4307f1_Datos_Etapa-1"** y es un poco mas manejable si lo dejamos como **"Datos_Etapa1"**

In [6]:
data_raw = pd.read_csv('./Data/Datos_Etapa1.csv',sep=',')
#datatest = pd.read_csv('./CopiaConVaciosDatos_Etapa-1.csv',sep=',')
#datatest.duplicated().sum()

In [7]:
#Verificamos que se encuentren cargados
data_raw.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


In [8]:
# Luego de verificar que se hayan cargado entonces  realizamos una copia del mismo para utilizarla durante todo el ciclo de ML
data = data_raw.copy()
data.head()

,season,weekday,weathersit,temp,atemp,hum,windspeed,cnt,time_of_day
0,Winter,6,Clear,3.28,3.0014,0.81,0.0,16,Night
1,Winter,6,Clear,2.34,1.9982,0.80,0.0,40,Night
2,Winter,6,Clear,2.34,1.9982,0.80,0.0,32,Night
3,Winter,6,Clear,3.28,3.0014,0.75,0.0,13,Night
4,Winter,6,Clear,3.28,3.0014,0.75,0.0,1,Night


## Verificación de datos
Es este paso realizaremos la verificación de los datos para saber si vienen completos, vacios, NaN y duplicados

In [9]:
# Con estas funciones podemos sumar la cantidad de True que contiene cada columna si dado el caso tine un dato en None, NaN o NaT o campos vacios
data.isna().sum()
#Al ejecutar vemos que no hay datos faltantes

season         0
weekday        0
weathersit     0
temp           0
atemp          0
hum            0
windspeed      0
cnt            0
time_of_day    0
dtype: int64

In [10]:
#Ahora revisaremos si existen duplicados
data.duplicated().sum()
#Al ejecutar la función vemos que existen 42 filas duplicadas

np.int64(42)

A continuación lo que deseo es ver si algún dato se encuentra corrompido por ejemplo:
- Season solo debería tener 4 unicos datos 'Winter', 'Spring', 'Summer', 'Fall' y no debería existir un dato corrompido como **Winterr** o **sumer**
- Tambíen deseo comprobar que los campos númericos solo tengan datos númericos, no deberíamos ver texto o caracteres alfabeticos.

Para el primer punto usaré `data.nunique()` que me sirve para ver los datos únicos de cada columna.

Para el segundo uso `data.dtypes` el cual me permite ver que tipo de dato tiene cada columna. 
- Si es Object entonces contiene caracteres alfabéticos.
- Si el dato es int o float entonces solo contiene  números.




In [28]:
data.nunique()

season           4
weekday          7
weathersit       4
temp            50
atemp           65
hum             89
windspeed       30
cnt            869
time_of_day      3
dtype: int64

Teniendo en cuenta el diccionario deberíamos ver datos de unicos de la siguiente manera:
- **Season: 4**  - Estación del año (Winter, Spring, Summer, Fall)
- **Weathersit: 4**  - Clima (Clear, Mist, Light Rain, Heavy Rain)
- **Time_of_day: 3**  - Parte del día (Morning, Evening, Night)
- **weekday: 7**  -  Día de la semana (de 1 a 7)

Al observar el resultado observamos que **todos los datos son correctos** 

> Ajuste de limpieza

In [27]:
data.dtypes

season          object
weekday          int64
weathersit      object
temp           float64
atemp          float64
hum            float64
windspeed      float64
cnt              int64
time_of_day     object
dtype: object

Revisando primeramente el diccionario deberíamos ver solo **Object** en los campos **season, weathersit, time_of_day** y en el resto de campos deberían ser un tipo de dato númerico.

Al ejecutar observamos que **todo está correcto** como se espera. 

In [26]:
data.nunique()

season           4
weekday          7
weathersit       4
temp            50
atemp           65
hum             89
windspeed       30
cnt            869
time_of_day      3
dtype: int64